In [ ]:
from io import StringIO

import altair as alt
import geopandas as gpd
import pandas as pd
from utils.geo import cria_mapa_com_graficos
alt.renderers.set_embed_options(format_locale="pt-BR", time_format_locale="pt-BR")


RendererRegistry.enable('default')

In [ ]:
# Lê o arquivo CSV removendo aspas duplas
# O CSV original é um CSV "sujo", criado com Excel, que adiciona aspas duplas
# em torno de cada campo, o que causa problemas na leitura.
with open(
    "../data/dados_exemplo_poluentes_no_acentos.csv", "r", encoding="utf-8-sig"
) as f:
    linhas = [linha.replace('"', "") for linha in f]

# Converte as linhas limpas para um buffer em memória
csv_buffer = StringIO("".join(linhas))

# Lê o CSV normalmente após a limpeza
df = pd.read_csv(csv_buffer, sep=",")
#
# Remove a coluna "unit" se todos os valores forem iguais
# Neste caso, todos os valores são "mg/L", então a coluna é desnecessária
if "unit" in df.columns and df["unit"].nunique() == 1:
    df = df.drop(columns=["unit"])

# Converte as colunas de latitude e longitude para o tipo numérico
df["lat"] = pd.to_numeric(df["lat"], errors="coerce")
df["lon"] = pd.to_numeric(df["lon"], errors="coerce")

# Remove Estacao do nome das estações e converte para categoria
df["station_name"] = df["station_name"].str.replace("Estacao", "")
df["station_name"] = df["station_name"].str.strip()
df["station_name"] = df["station_name"].astype("category")

# Converte a coluna de data para o tipo datetime
df["sample_dt"] = pd.to_datetime(df["sample_dt"], format="%Y-%m-%d")

# Remove a coluna station_id, já que station_name é suficiente para identificar os pontos de coleta
# Cada station_id tem um station_name único
df = df.drop(columns=["station_id"])

# Transforma o DataFrame para o formato longo (long format) para facilitar a plotagem
# com bibliotecas de visualização
df_longo = df.melt(
    id_vars=["station_name", "lat", "lon", "sample_dt"],
    value_vars=["pol_a", "pol_b"],
    var_name="pollutant",
    value_name="value",
)

gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df["lon"], df["lat"]), crs="EPSG:4326"
)

gdf_longo = gpd.GeoDataFrame(
    df_longo,
    geometry=gpd.points_from_xy(df_longo["lon"], df_longo["lat"]),
    crs="EPSG:4326",
)

In [ ]:
df_longo_formatado = df_longo.copy(deep=True)
df_longo_formatado["pollutant"] = df_longo_formatado["pollutant"].replace(
    {"pol_a": "A", "pol_b": "B"}
)

source = df_longo_formatado[df_longo["station_name"] == "M1"]

points = (
    alt.Chart(source)
    .mark_line(point=alt.OverlayMarkDef(filled=False, fill="white"))
    .encode(
        x=alt.X("sample_dt:T", axis=alt.Axis(format="%d %b", title="Data")),
        y=alt.Y("value:Q", title="Valor (mg/L)"),
        color=alt.Color(
            "pollutant:N",
            legend=alt.Legend(title="Poluente"),
            scale=alt.Scale(range=["green", "purple"]),
        ),
        tooltip=[
            alt.Tooltip("station_name:N", title="Estação"),
            alt.Tooltip("sample_dt:T", title="Data da coleta", format="%d-%m-%Y"),
            alt.Tooltip("value:Q", title="Valor (mg/L)"),
        ],
    )
).add_params()


points


alt.Chart(...)